# Set up BEIR

In [ ]:
!nvidia-smi

Fri Mar 29 13:54:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install beir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 722.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━

In [ ]:
from beir import util, LoggingHandler

import logging
import pathlib, os

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

/usr/local/lib/python3.10/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Setup FinBert

In [ ]:
!pip install transformers numpy torch

In [ ]:
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from transformers import AutoModel, AutoTokenizer
import numpy as np
import torch
from tqdm import trange
import os
from typing import List, Dict

class FinBERT:
    def __init__(self, model_path: str, device, **kwargs):
        self.device = device
        self.bert_q = AutoModel.from_pretrained(model_path)
        # self.bert_q = AutoModel.from_pretrained(model_path, num_labels=3)
        self.bert_q.eval()
        self.bert_q.to(self.device)

        self.bert_d = AutoModel.from_pretrained(model_path)
        # self.bert_d = AutoModel.from_pretrained(model_path, num_labels=3)
        self.bert_d.eval()
        self.bert_d.to(self.device)

        self.tokenizer = AutoTokenizer.from_pretrained(model_path)


    def encode_queries(self, queries: List[str], batch_size: int = 16, **kwargs) -> np.ndarray:
        query_embeddings = []

        with torch.no_grad():
          for start_idx in trange(0, len(queries), batch_size):
            encoded = self.tokenizer(queries[start_idx:start_idx+batch_size], truncation=True, padding=True, return_tensors='pt', max_length=512)
            encoded.to(self.device)
            model_out = self.bert_q(**encoded)
            query_embeddings += model_out.last_hidden_state[:, 0, :].detach().cpu()

        return torch.stack(query_embeddings)

    def encode_corpus(self, corpus: List[Dict[str, str]], batch_size: int, **kwargs) -> np.ndarray:
        corpus_embeddings = []

        with torch.no_grad():
          for start_idx in trange(0, len(corpus), batch_size):
            titles = [row['title'] for row in corpus[start_idx: start_idx + batch_size]]
            texts = [row['text']  for row in corpus[start_idx: start_idx + batch_size]]
            encoded = self.tokenizer(titles, texts, truncation='longest_first', padding=True, return_tensors='pt', max_length=512)
            encoded.to(self.device)
            model_out = self.bert_d(**encoded)
            corpus_embeddings += model_out.last_hidden_state[:, 0, :].detach().cpu()

        return torch.stack(corpus_embeddings)

#Setup Datasets

In [ ]:
import pathlib, os
from beir import util

def download_dataset(dataset_name: str):
  url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
  out_dir = os.path.join(os.getcwd(), "datasets")
  data_path = util.download_and_unzip(url, out_dir)
  print("Dataset downloaded here: {}".format(data_path))
  return data_path

In [ ]:
#!ls datasets/scifact/

In [ ]:
from beir.datasets.data_loader import GenericDataLoader


#dataset = "scifact"
dataset = "fiqa"
#dataset="trec-covid"
data_path = download_dataset(dataset_name=dataset)
corpus, queries, qrels = GenericDataLoader(data_path).load(split="test") # or split = "train" or "dev"

/content/datasets/trec-covid.zip:   0%|          | 0.00/70.5M [00:00<?, ?iB/s]

Dataset downloaded here: /content/datasets/trec-covid


  0%|          | 0/171332 [00:00<?, ?it/s]

# Evaluate

In [ ]:
from beir.retrieval.evaluation import EvaluateRetrieval

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

finbert = DRES(FinBERT(model_path="yiyanghkust/finbert-tone", device=device), batch_size=16)
retriever = EvaluateRetrieval(finbert, score_function="dot")

results = retriever.retrieve(corpus, queries)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

100%|██████████| 1334/1334 [00:28<00:00, 46.45it/s]


In [ ]:
ndcg, _map, recall, precision = EvaluateRetrieval.evaluate(qrels, results, retriever.k_values)
ndcg, _map, recall, precision

({'NDCG@1': 0.0,
  'NDCG@3': 0.01061,
  'NDCG@5': 0.01468,
  'NDCG@10': 0.01359,
  'NDCG@100': 0.00852,
  'NDCG@1000': 0.01057},
 {'MAP@1': 0.0,
  'MAP@3': 3e-05,
  'MAP@5': 8e-05,
  'MAP@10': 0.00011,
  'MAP@100': 0.00017,
  'MAP@1000': 0.00029},
 {'Recall@1': 0.0,
  'Recall@3': 6e-05,
  'Recall@5': 0.00031,
  'Recall@10': 0.00055,
  'Recall@100': 0.00194,
  'Recall@1000': 0.01097},
 {'P@1': 0.0,
  'P@3': 0.01333,
  'P@5': 0.02,
  'P@10': 0.018,
  'P@100': 0.0094,
  'P@1000': 0.00572})

In [ ]:
mrr = retriever.evaluate_custom(qrels, results, retriever.k_values, metric="mrr")
recall_cap = retriever.evaluate_custom(qrels, results, retriever.k_values, metric="recall_cap")
hole = retriever.evaluate_custom(qrels, results, retriever.k_values, metric="hole")
top_k_accuracy = retriever.evaluate_custom(qrels, results, retriever.k_values, metric="top_k_accuracy")
mrr, recall_cap, hole, top_k_accuracy